In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import yfinance as yf
from requests.exceptions import HTTPError
import pandas as pd

# List of ticker symbols
ticker_symbols = [
    "RELIANCE.NS",
    "TCS.NS",
    "HDB",
    "IBN",
    "INFY",
    "SBIN.NS",
    "LICI.NS",
    "BHARTIARTL.NS",
    "HINDUNILVR.NS",
    "ITC.NS",
    "LT.NS"
]


# Counters
total_companies = len(ticker_symbols)
discarded_companies = 0
available_companies = 0

# Set start and end dates
start_date = '2014-01-01'
end_date = '2024-01-01'

# Create an empty DataFrame to store data for companies with sector and industry
all_stock_data = pd.DataFrame()

# Loop through each ticker symbol
for ticker_symbol in ticker_symbols:
    try:
        # Create a Ticker object
        ticker = yf.Ticker(ticker_symbol)

        # Get the info dictionary containing various information including sector and industry
        info = ticker.info

        # Get sector and industry information
        sector = info.get('sector')
        industry = info.get('industry')

        # Check if both sector and industry are not None
        if sector is not None and industry is not None:
            # Download historical data for the current ticker symbol
            stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

            # Add stock information as columns to the DataFrame
            stock_data['Company Name'] = info['longName']
            stock_data['Sector'] = sector
            stock_data['Industry'] = industry

            # Append the stock data to the DataFrame for all stocks
            all_stock_data = pd.concat([all_stock_data, stock_data])

            available_companies += 1
        else:
            discarded_companies += 1
    except HTTPError as e:
        if e.response.status_code == 404:
            print(f"Error 404: Ticker {ticker_symbol} not found. Discarding...")
            discarded_companies += 1

# Print summary
print("Summary:")
print("Total companies input:", total_companies)
print("Number of companies discarded:", discarded_companies)
print("Number of companies available with sector and industry:", available_companies)

# Display the DataFrame
print("\nStock Data:")
print(all_stock_data)


In [ ]:
df = pd.DataFrame(all_stock_data)
df

In [ ]:
df.rename(columns={'Company Name': 'company'}, inplace=True)

In [ ]:
#pd.set_option('display.max_rows', None)
company_counts=df['company_counts'] = df['company'].value_counts()
company_counts


In [ ]:
num_companies_with_2465 = (company_counts == 2465).sum()
num_companies_with_2465


In [ ]:
df['days_count'] = df['company'].map(df['company'].value_counts())
df

In [ ]:
selected_companies = []


In [ ]:
for company, days_count in df.groupby('company')['days_count'].first().items():
    if days_count == 2465:
        selected_companies.append(company)



In [ ]:
df1 = df[df['company'].isin(selected_companies)]
df1

In [ ]:
df1.drop(columns=['company_counts'], inplace=True)

In [ ]:
import ta

import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns



In [ ]:
List_of_companies = list(set(df1['company'].unique()))

In [ ]:
All_df_list = []
for comp in List_of_companies:
#     print(comp)
    Individual_comp = df1.loc[df1['company'] == comp]
    Individual_comp = Individual_comp.sort_values('Date')
    Individual_comp['prev close'] = Individual_comp['Adj Close'].shift(1)
    Individual_comp['Returns'] = (Individual_comp['Adj Close'] - Individual_comp['prev close'])/Individual_comp['prev close']
    All_df_list.append(Individual_comp)
final_df = pd.concat(All_df_list)
final_df

In [ ]:
df2=final_df.pivot(columns="company",values="Returns")

In [ ]:
df2

In [ ]:
# Get unique sectors
sectors = df1["Sector"].unique()

# Create an empty dictionary to store sector-wise company names
sector_companies = {}

# Iterate over each sector
for sector in sectors:
    # Filter data for the current sector
    sector_data = df1[df1["Sector"] == sector]

    # Get unique company names within the sector
    companies = sector_data["company"].unique().tolist()

    # Store the list of company names in the dictionary
    sector_companies[sector] = companies

sector_companies

In [ ]:
data2=final_df.pivot(columns="company",values="Adj Close")
data2

In [ ]:
import pandas as pd

# Assuming df contains your data with columns 'sector', 'company', and 'adj close'

# Pivot the data to get adjusted close values for each company within each sector
sector_adj_close = df1.pivot_table(index='Date',columns= 'Sector', values='Adj Close')

# Display the resulting DataFrame
sector_adj_close


In [ ]:
# mean variance 
import pandas as pd
from pypfopt import expected_returns, risk_models
from pypfopt.efficient_frontier import EfficientFrontier

# Assuming selected_data contains your data with columns 'company', 'Sector', and 'Adj Close'

# Get unique sectors and create a dictionary to store sector-wise company names
sectors = df1["Sector"].unique()
sector_companies = {}
for sector in sectors:
    sector_data = df1[df1["Sector"] == sector]
    companies = sector_data["company"].unique().tolist()
    sector_companies[sector] = companies

# Define sector mappings based on the sector-wise company names dictionary
sector_mapper = {company: sector for sector, companies in sector_companies.items() for company in companies}

# Define sector constraints based on the sector-wise company names dictionary
sector_lower = {sector: 0 for sector in sectors}
sector_upper = {sector: 0.1 for sector in sectors}

# Calculate expected returns and the covariance matrix of the portfolio
mu = expected_returns.mean_historical_return(data2)
S = risk_models.sample_cov(data2)

# Create the Efficient Frontier Object
ef = EfficientFrontier(mu, S, solver='ECOS')

# Add sector constraints to the Efficient Frontier
ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)

# Optimize for the maximum Sharpe ratio
weights = ef.max_sharpe()

# Clean the raw weights and print them
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

# Calculate and print the portfolio performance
ef.portfolio_performance(verbose=True)


In [ ]:
import plotly.express as px

# Filter out zero-weighted assets
non_zero_weights = {asset: weight for asset, weight in cleaned_weights.items() if weight != 0}

# Extract labels and values from non-zero weights
labels = list(non_zero_weights.keys())
values = list(non_zero_weights.values())

# Create a pie chart using Plotly
fig = px.pie(names=labels, values=values, title='Non-zero Weight Portfolio Composition')
fig.show()


In [ ]:
# EFFICIENT CVAR
from pypfopt import EfficientCVaR

# Calculate expected returns and the covariance matrix of the portfolio
mu = expected_returns.mean_historical_return(data2)
S = risk_models.sample_cov(data2)

# Create the Efficient Frontier Object with CVaR
ef = EfficientCVaR(mu, S)

# Add sector constraints to the Efficient Frontier
ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)

# Optimize for the minimum CVaR
weights = ef.min_cvar()

# Clean the raw weights and print them
cleaned_weightss = ef.clean_weights()
print(cleaned_weightss)



# Calculate and print the portfolio performance
ef.portfolio_performance(verbose=True)

In [ ]:
import plotly.express as px

# Filter out zero-weighted assets
non_zero_weightss = {asset: weight for asset, weight in cleaned_weightss.items() if weight != 0}

# Extract labels and values from non-zero weights
labels = list(non_zero_weightss.keys())
values = list(non_zero_weightss.values())

# Create a pie chart using Plotly
fig = px.pie(names=labels, values=values, title='Non-zero Weight Portfolio Composition')
fig.show()


# SECTOR WISE

In [ ]:
# mean variance 
import pandas as pd
from pypfopt import expected_returns, risk_models
from pypfopt.efficient_frontier import EfficientFrontier

# Assuming selected_data contains your data with columns 'company', 'Sector', and 'Adj Close'

# Get unique sectors and create a dictionary to store sector-wise company names
sectors = df1["Sector"].unique()
sector_companies = {}
for sector in sectors:
    sector_data = df1[df1["Sector"] == sector]
    companies = sector_data["company"].unique().tolist()
    sector_companies[sector] = companies

# Define sector mappings based on the sector-wise company names dictionary
sector_mapper = {company: sector for sector, companies in sector_companies.items() for company in companies}

# Define sector constraints based on the sector-wise company names dictionary
sector_lower = {sector: 0 for sector in sectors}
sector_upper = {sector: 0.1 for sector in sectors}

# Calculate expected returns and the covariance matrix of the portfolio
mu = expected_returns.mean_historical_return(sector_adj_close)
S = risk_models.sample_cov(sector_adj_close)

# Create the Efficient Frontier Object
ef = EfficientFrontier(mu, S, solver='ECOS')

# Add sector constraints to the Efficient Frontier
ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)

# Optimize for the maximum Sharpe ratio
weights = ef.max_sharpe()

# Clean the raw weights and print them
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

# Calculate and print the portfolio performance
ef.portfolio_performance(verbose=True)


In [ ]:
import plotly.express as px

# Filter out zero-weighted assets
non_zero_weights = {asset: weight for asset, weight in cleaned_weights.items() if weight != 0}

# Extract labels and values from non-zero weights
labels = list(non_zero_weights.keys())
values = list(non_zero_weights.values())

# Create a pie chart using Plotly
fig = px.pie(names=labels, values=values, title='Non-zero Weight Portfolio Composition')
fig.show()


In [ ]:
# EFFICIENT CVAR
from pypfopt import EfficientCVaR

# Calculate expected returns and the covariance matrix of the portfolio
mu = expected_returns.mean_historical_return(sector_adj_close)
S = risk_models.sample_cov(sector_adj_close)

# Create the Efficient Frontier Object with CVaR
ef = EfficientCVaR(mu, S, beta= 0.70)

# Add sector constraints to the Efficient Frontier
ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)

# Optimize for the minimum CVaR
weights = ef.min_cvar()

# Clean the raw weights and print them
cleaned_weightss = ef.clean_weights()
print(cleaned_weightss)



# Calculate and print the portfolio performance
ef.portfolio_performance(verbose=True)

In [ ]:
import plotly.express as px

# Filter out zero-weighted assets
non_zero_weightss = {asset: weight for asset, weight in cleaned_weightss.items() if weight != 0}

# Extract labels and values from non-zero weights
labels = list(non_zero_weightss.keys())
values = list(non_zero_weightss.values())

# Create a pie chart using Plotly
fig = px.pie(names=labels, values=values, title='Non-zero Weight Portfolio Composition')
fig.show()


In [ ]:
'''import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Assuming `df1` is your DataFrame containing stock data
List_of_companies = list(set(df1['company'].unique()))

All_df_list = []
for comp in List_of_companies:
    Individual_comp = df1.loc[df1['company'] == comp]
    Individual_comp = Individual_comp.sort_values('Date')
    Individual_comp['prev close'] = Individual_comp['Adj Close'].shift(1)
    Individual_comp['Returns'] = (Individual_comp['Adj Close'] - Individual_comp['prev close']) / Individual_comp[
        'prev close']
    All_df_list.append(Individual_comp)
final_df = pd.concat(All_df_list)

# Pivot the DataFrame to get returns for each company
returns_df = final_df.pivot(columns="company", values="Returns")

# Remove rows with NaN or infinite values
clean_returns_df = returns_df.dropna().replace([np.inf, -np.inf], np.nan).dropna()

# Calculate expected returns and sample covariance matrix of asset returns
mu = expected_returns.mean_historical_return(clean_returns_df)
S = risk_models.sample_cov(clean_returns_df)

# Initialize EfficientFrontier
ef = EfficientFrontier(mu, S)

# Optimize for maximum Sharpe ratio
weights = ef.max_sharpe()

# Clean the weights
cleaned_weights = ef.clean_weights()

# Display the cleaned weights
print(cleaned_weights)
'''

In [ ]:
'''import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Step 1: Calculate expected returns and covariance matrix
returns = df2
mean_returns = returns.mean()
cov_matrix = returns.cov()

# Step 2: Instantiate EfficientFrontier object with a lower risk-free rate value and specify the solver
ef = EfficientFrontier(mean_returns, cov_matrix, solver='ECOS')

# Step 3: Define objective and constraints (optional)
# Here we'll use default constraints (weights between 0 and 1, sum of weights = 1)

# Step 4: Optimize for maximum Sharpe ratio with a lower risk-free rate value
#risk_free_rate = 0.0001  # Adjust the risk-free rate value as needed
weights_max_sharpe = ef.max_sharpe(risk_free_rate)
weigg]
# Print optimized weights for minimum volatility and maximum Sharpe ratio
print("Weights for minimum volatility portfolio:")
print(weights_min_volatility)
print("\nWeights for maximum Sharpe ratio portfolio:")
print(weights_max_sharpe)

'''

In [ ]:
'''import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import plotly.graph_objects as go

# Step 1: Calculate expected returns and covariance matrix
returns = df2
mean_returns = returns.mean()
cov_matrix = returns.cov()

# Step 2: Instantiate EfficientFrontier object with a lower risk-free rate value and specify the solver
ef = EfficientFrontier(mean_returns, cov_matrix, solver='ECOS')

# Step 3: Define objective and constraints (optional)
# Here we'll use default constraints (weights between 0 and 1, sum of weights = 1)

# Step 4: Optimize for maximum Sharpe ratio with a lower risk-free rate value
#risk_free_rate = 0.0001  # Adjust the risk-free rate value as needed
weights_max_sharpe = ef.max_sharpe(risk_free_rate)

# Create a Pie Chart using Plotly
fig = go.Figure(data=[go.Pie(labels=weights_max_sharpe.keys(), values=weights_max_sharpe.values())])

# Set layout options
fig.update_layout(title='Optimized Weights for Maximum Sharpe Ratio Portfolio')

# Show the plot
fig.show()'''


In [ ]:
# Monte Carlo

In [ ]:
'''import numpy as np
np.random.seed(1)
# Weight each security
weights = np.random.random((389,1))
# normalize it, so that some is one
weights /= np.sum(weights)
print(f'Normalized Weights : {weights.flatten()}')

# We generally do log return instead of return
log_ret = np.log(data2/ data2.shift(1))
log_ret

# Expected return (weighted sum of mean returns). Mult by 252 as we always do annual calculation and year has 252 business days
exp_ret = log_ret.mean().dot(weights)*252
print(f'\nExpected return of the portfolio is : {exp_ret[0]}')
# Exp Volatility (Risk)
exp_vol = np.sqrt(weights.T.dot(252*log_ret.cov().dot(weights)))
print(f'\nVolatility of the portfolio: {exp_vol[0][0]}')

# Sharpe ratio
sr = exp_ret / exp_vol
print(f'\nSharpe ratio of the portfolio: {sr[0][0]}')'''

In [ ]:
''''# number of simulation
#n = 50_000
n = 10

port_weights = np.zeros(shape=(n,len(data2.columns)))
port_volatility = np.zeros(n)
port_sr = np.zeros(n)
port_return = np.zeros(n)

num_securities = len(data2.columns)
# num_securities
for i in range(n):
    # Weight each security
    weights = np.random.random(389)
    # normalize it, so that some is one
    weights /= np.sum(weights)
    port_weights[i,:] = weights
    #     print(f'Normalized Weights : {weights.flatten()}')

    # Expected return (weighted sum of mean returns). Mult by 252 as we always do annual calculation and year has 252 business days
    exp_ret = log_ret.mean().dot(weights)*252
    port_return[i] = exp_ret
#     print(f'\nExpected return is : {exp_ret[0]}')

    # Exp Volatility (Risk)
    exp_vol = np.sqrt(weights.T.dot(252*log_ret.cov().dot(weights)))
    port_volatility[i] = exp_vol
#     print(f'\nVolatility : {exp_vol[0][0]}')

    # Sharpe ratio
    sr = exp_ret / exp_vol
    port_sr[i] = sr
#   print(f'\nSharpe ratio : {sr[0][0]}')''''

In [ ]:
'''# Index of max Sharpe Ratio
max_sr = port_sr.max()
ind = port_sr.argmax()
# Return and Volatility at Max SR
max_sr_ret = port_return[ind]
max_sr_vol = port_volatility[ind]'''

In [ ]:
'''plt.figure(figsize=(20,15))
plt.scatter(port_volatility,port_return,c=port_sr, cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility', fontsize=15)
plt.ylabel('Return', fontsize=15)
plt.title('Efficient Frontier (Bullet Plot)', fontsize=15)
plt.scatter(max_sr_vol, max_sr_ret, c='blue', s=150, edgecolors='red', marker='o', label='Max \
Sharpe ratio Portfolio')
plt.legend();'''


In [ ]:
'''for weight, company in zip(port_weights[ind],company):
    print(f'{round(weight * 100, 2)} % of {company} should be bought.')

# best portfolio return
print(f'\nMarkowitz optimal portfolio return is : {round(max_sr_ret * 100, 2)}% with volatility \
{max_sr_vol}')'''

In [ ]:
'''from scipy import optimize'''

In [ ]:
'''log_mean = log_ret.mean() * 252
cov = log_ret.cov() * 252'''

In [ ]:
''''# Some helper functions
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = log_mean.dot(weights)
    vol = np.sqrt(weights.T.dot(cov.dot(weights)))
    sr = ret / vol
    return np.array([ret, vol, sr])

# Negate Sharpe ratio as we need to max it but Scipy minimize the given function
def neg_sr(weights):
    return get_ret_vol_sr(weights)[-1] * -1

# check sum of weights
def check_sum(weights):
    return np.sum(weights) - 1

# Constraints for the optimization problem
cons = {'type':'eq','fun':check_sum}
# bounds on weights
bounds = [(0,1)] * 389

# initial guess for optimization to start with
init_guess = [.25 for _ in range(389)]


# Call minimizer
opt_results = optimize.minimize(neg_sr, init_guess, constraints=cons, bounds=bounds, method='SLSQP')''''

In [ ]:
'''optimal_weights = opt_results.x
# optimal_weights
for st, i in zip(company,optimal_weights):
    print(f'Stock {st} has weight {np.round(i*100,2)} %')'''

In [ ]:
'''mc_weights = port_weights[ind]
for st, i in zip(company,mc_weights):
    print(f'Stock {st} has weight {np.round(i*100,2)} %')'''

In [ ]:
'''# Comparing two results we see that we get very close results
(optimal_weights - mc_weights)'''

In [ ]:
'''get_ret_vol_sr(optimal_weights), get_ret_vol_sr(mc_weights)

print('For a given portfolio we have: (Using SciPy optimizer)\n \n')
for i, j in enumerate('Return Volatility SharpeRatio'.split()):
    print(f'{j} is : {get_ret_vol_sr(optimal_weights)[i]}\n')

print('For a given portfolio we have: (Using Monte Carlo)\n \n')
for i, j in enumerate('Return Volatility SharpeRatio'.split()):
    print(f'{j} is : {get_ret_vol_sr(mc_weights)[i]}\n')'''

In [ ]:
'''frontier_y = np.linspace(port_return.min(), port_return.max(), 10)
frontier_vol = []

def minimize_vol(weights):
    return get_ret_vol_sr(weights)[1]

for possible_ret in frontier_y:
    cons = ({'type':'eq','fun':check_sum},
            {'type':'eq','fun':lambda w:get_ret_vol_sr(w)[0] - possible_ret})
    result = optimize.minimize(minimize_vol, init_guess, method='SLSQP', constraints=cons, bounds=bounds)
    frontier_vol.append(result['fun']) '''

In [ ]:
'''plt.figure(figsize=(20,15))
plt.scatter(port_volatility,port_return,c=port_sr, cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility', fontsize=15)
plt.ylabel('Return', fontsize=15)
plt.title('Efficient Frontier', fontsize=15)
plt.scatter(max_sr_vol, max_sr_ret, c='blue', s=150, edgecolors='red', marker='o')

plt.plot(frontier_vol, frontier_y, c='magenta', ls='--', lw=3, label='Efficient Frontier')
plt.legend();'''

In [ ]:
''''import numpy as np
from scipy.optimize import minimize

# Sample stock returns data
returns = df2

# Calculate the covariance matrix
cov_matrix = np.cov(returns)

def risk_contribution(weights, cov_matrix):
    port_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    risk_contributions = weights * np.dot(cov_matrix, weights) / port_variance
    return risk_contributions

def target_risk(weights, cov_matrix, target_risk):
    return np.sum(np.square(risk_contribution(weights, cov_matrix)) - target_risk)

def calculate_weights(cov_matrix, target_risk):
    num_assets = cov_matrix.shape[0]
    init_weights = np.ones(num_assets) / num_assets
    bounds = ((0, 1),) * num_assets  # Bounds for weights (0 to 1)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Sum of weights equal to 1
    optimized = minimize(target_risk, init_weights, args=(cov_matrix, target_risk),
                         bounds=bounds, constraints=constraints)
    return optimized.x

# Target risk (adjust according to your preferences)
target_risk = 0.03

# Calculate weights using risk parity method
weights = calculate_weights(cov_matrix, target_risk)

print("Risk Parity Weights:", weights)
print("Risk Contributions:", risk_contribution(weights, cov_matrix))
''''

In [ ]:
'''import QuantLib as ql
import numpy as np
from scipy.optimize import minimize

# Sample data
returns = df2
# Calculate covariance matrix
cov_matrix = np.cov(returns)

# Define the objective function
def objective(weights, cov_matrix):
    port_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    return port_variance

# Define the constraint (sum of weights equals 1)
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Define initial guess for weights
init_weights = np.ones(len(returns)) / len(returns)

# Optimize using SciPy
result = minimize(objective, init_weights, args=(cov_matrix,), constraints=constraints)

# Extract optimized weights
optimal_weights = result.x

print("Optimized Weights:", optimal_weights)'''


In [ ]:
plt.figure(figsize=(14, 7))
for c in data2.columns.values:
    plt.plot(data2.index, data2[c], lw=3, alpha=0.8,label=c)
plt.legend(loc='upper left', fontsize=9)
plt.ylabel('price in Rs')

def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

def random_portfolios(num_portfolios, mean_returns, cov_matrix,risk_free_rate):
    results = np.zeros((3, num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_return = np.sum(mean_returns * weights)
        portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record
returns = data2.pct_change()
mean_returns = returns.mean()
cov_matrix = returns.cov()
num_portfolios = 25000
risk_free_rate = 0.0178

def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, weights = random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate)
    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[0, max_sharpe_idx], results[1, max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx], index=data2.columns, columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100, 2) for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    
    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0, min_vol_idx], results[1, min_vol_idx]
    min_vol_allocation = pd.DataFrame(weights[min_vol_idx], index=data2.columns, columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100, 2) for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print("-"*80)
    print("Maximum Sharpe Ratio Portfolio Allocation\n")
    print("Annualised Return:", round(rp, 2))
    print("Annualised Volatility:", round(sdp, 2))
    print("\n")
    print(max_sharpe_allocation)
    print("-"*80)
    print("Minimum Volatility Portfolio Allocation\n")
    print("Annualised Return:", round(rp_min, 2))
    print("Annualised Volatility:", round(sdp_min, 2))
    print("\n")
    print(min_vol_allocation)
# display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)


In [ ]:
output=pd.DataFrame(display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios,risk_free_rate))
output

In [ ]:
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_var

def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [ ]:
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)

    return result

In [ ]:
def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    def portfolio_return(weights):
        return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[1]

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result


def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

In [ ]:
def display_calculated_ef_with_random(mean_returns, cov_matrix,num_portfolios, risk_free_rate):
    results, _ = random_portfolios(num_portfolios,mean_returns,cov_matrix, risk_free_rate)
    
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'], mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=table.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    max_sharpe_allocation

    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=table.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    
    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')

    target = np.linspace(rp_min, 0.32, 50)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    plt.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
    plt.title('Calculated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)

In [ ]:
display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Define your utility functions here
table=data2
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_var

def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)

    return result

def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    def portfolio_return(weights):
        return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[1]

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    result = minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

def display_calculated_ef_with_random(mean_returns, cov_matrix,num_portfolios, risk_free_rate):
    results, _ = random_portfolios(num_portfolios,mean_returns,cov_matrix, risk_free_rate)
    
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'], mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=table.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    max_sharpe_allocation

    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=table.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    
    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')

    target = np.linspace(rp_min, 0.32, 50)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    plt.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
    plt.title('Calculated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)
    plt.show()

# Example usage:
# display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)


In [ ]:
display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)

In [ ]:
'''import pandas as pd

# Assuming your data is stored in a DataFrame named 'df1' with a column 'Sector' indicating the sector of each stock

# Extract unique sectors from your data
unique_sectors = df1['Sector'].unique()

# Define empty dictionaries to store sector mappings and constraints
sector_mapper = {}
sector_lower = {}
sector_upper = {}

# Assign sectors to sector_mapper and set initial constraints
for sector in unique_sectors:
    sector_mapper.update({stock: sector for stock in df1[df1['Sector'] == sector].index})
    sector_lower[sector] = 0  # at least 0% allocation to the sector initially
    sector_upper[sector] = 1  # at most 100% allocation to the sector initially

# Update constraints if needed (e.g., you want to set specific lower and upper bounds for each sector)

# Print the sector mappings and constraints
print("Sector Mapper:")
print(sector_mapper)
print("\nSector Lower Bounds:")
print(sector_lower)
print("\nSector Upper Bounds:")
print(sector_upper)'''


In [ ]:
# # DISCRETE ALLOCATION OF EFFICIENT FRONTIER
# from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
# portfolio_amount = float(input("Enter the amount you want to invest: "))
# if portfolio_amount != '' :
#     # Get discrete allocation of each share per stock

#     latest_prices = get_latest_prices(df)
#     weights = cleaned_weights
#     discrete_allocation = DiscreteAllocation(weights, latest_prices , total_portfolio_value = int(portfolio_amount))
#     allocation , leftover = discrete_allocation.lp_portfolio()

#     discrete_allocation_list = []


#     for symbol in allocation:
#         discrete_allocation_list.append(allocation.get(symbol))


#     portfolio_df = pd.DataFrame(columns =['Ticker' , 'Number of stocks to buy'])

#     portfolio_df['Ticker'] = allocation
#     portfolio_df['Number of stocks to buy'] = discrete_allocation_list
#     print('Number of stocks to buy with the amount of ₨ ' + str(portfolio_amount))
#     print(portfolio_df)
#     print('Funds remaining with you will be: ₨' , int(leftover))


In [ ]:
# SHARPE RATIO FOR EFFICIENT CVAR
from pypfopt import expected_returns, risk_models
from pypfopt import EfficientCVaR
import pandas as pd

# Assume you have already defined mu, S, and ef as per your code snippet

# Calculate the expected returns and the covariance matrix of the portfolio
mu = expected_returns.mean_historical_return(data2)
S = risk_models.sample_cov(data2)

# Create the Efficient Frontier Object with CVaR
ef = EfficientCVaR(mu, S)

# Add sector constraints to the Efficient Frontier
ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)

# Optimize for the minimum CVaR
weights = ef.min_cvar()

# Clean the raw weights and print them
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

# Calculate the portfolio performance
# Assuming you have risk-free rate and daily returns of the portfolio available
# Replace rf_rate with the actual risk-free rate
rf_rate = 0.02 / 252  # Assuming an annual risk-free rate of 2%, converted to daily rate
portfolio_returns = ef.portfolio_performance()[0]  # Portfolio returns
portfolio_volatility = ef.portfolio_performance()[1]  # Portfolio volatility (standard deviation)

# Calculate Sharpe Ratio
sharpe_ratio = (portfolio_returns - rf_rate) / portfolio_volatility

# Print the Sharpe Ratio
print("Sharpe Ratio:", sharpe_ratio)

In [ ]:
# # DISCRETE ALLOCATION OF EFFICIENT CVAR
# from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
# portfolio_amount = float(input("Enter the amount you want to invest: "))
# if portfolio_amount != '' :
#     # Get discrete allocation of each share per stock

#     latest_prices = get_latest_prices(df)
#     weights = cleaned_weights
#     discrete_allocation = DiscreteAllocation(weights, latest_prices,total_portfolio_value = int(portfolio_amount))
#     allocation , leftover = discrete_allocation.lp_portfolio()

#     discrete_allocation_list = []


#     for symbol in allocation:
#         discrete_allocation_list.append(allocation.get(symbol))


#     portfolio_df = pd.DataFrame(columns =['Ticker' , 'Number of stocks to buy'])

#     portfolio_df['Ticker'] = allocation
#     portfolio_df['Number of stocks to buy'] = discrete_allocation_list
#     print('Number of stocks to buy with the amount of ₨ ' + str(portfolio_amount))
#     print(portfolio_df)
#     print('Funds remaining with you will be: ₨' , int(leftover))


In [ ]:
'''import matplotlib.pyplot as plt

#Your portfolio weights
weights = {
   'Reliance': 0.11172,
   'ONGC': 0.03206,
   'Adani': 0.16544,
    'Asian Paints': 0.21898,
    'UltraTech': 0.08102,
    'Maruti Suzuki': 0.1,
    'Tata Motors': 0.0,
    'HCL': 0.21405,
    'TCS': 0.08595,
    'Infosys': 0.0
}

#The values are your portfolio weights
values = list(weights.values())

#The labels are the stock names
labels = list(weights.keys())

#Create the pie chart
plt.figure(figsize=(10, 6))
plt.pie(values, labels=labels, autopct='%1.1f%%')

#Display the plot
plt.show()'''


REBALANCING

In [ ]:
non_zero_weights

In [ ]:
# List of companies
companies = [
    'APAR Industries Limited',
    'APL Apollo Tubes Limited',
    'Abbott India Limited',
    'Adani Enterprises Limited',
    'Bajaj Finance Limited',
    'Bajaj Holdings & Investment Limited',
    'Bharti Airtel Limited',
    'Britannia Industries Limited',
    'Cantabil Retail India Limited',
    'Grindwell Norton Limited',
    'Hindustan Petroleum Corporation Limited',
    'Info Edge (India) Limited',
    'JBM Auto Limited',
    'KNR Constructions Limited',
    'Kingfa Science & Technology (India) Limited',
    'MPS Limited',
    'Muthoot Finance Limited',
    'Olectra Greentech Limited',
    'Persistent Systems Limited',
    'Petronet LNG Limited',
    'Power Grid Corporation of India Limited',
    'Procter & Gamble Hygiene and Health Care Limited',
    'Ratnamani Metals & Tubes Limited',
    'Reliance Industries Limited',
    'SJVN Limited',
    'Solar Industries India Limited',
    'Tanla Platforms Limited',
    'Tata Communications Limited',
    'The Phoenix Mills Limited',
    'Torrent Pharmaceuticals Limited',
    'Uno Minda Limited'
]

# List of weights
weights = [
    0.01873,
    0.04668,
    0.09293,
    0.02688,
    0.07044,
    0.02727,
    0.00171,
    0.04585,
    0.02809,
    0.03389,
    0.00433,
    0.04472,
    0.03117,
    0.00807,
    0.0122,
    0.02347,
    0.00229,
    0.03932,
    0.04366,
    0.01083,
    0.06563,
    0.05415,
    0.02094,
    0.0169,
    0.03437,
    0.02018,
    0.05634,
    0.02551,
    0.04566,
    0.00707,
    0.04074
]

In [ ]:
benchmark = '^BSESN'

In [ ]:
# Filter df1 based on the list of companies
filtered_df = df1[df1['company'].isin(companies)]

# Display the filtered DataFrame
print(filtered_df)

In [ ]:
filtered_df

In [ ]:
new1=filtered_df.pivot_table(index='Date',columns= 'company', values='Adj Close')
new1

In [ ]:
shares_df = pd.DataFrame(index=[new1.index[0]])
portfolio_value = 10**7
for s,w in zip(companies, weights):
    shares_df[s + '_shares'] = np.floor((portfolio_value * np.array(w)) / new1[s][0])

shares_df  

In [ ]:
# REBALANCING ENGINE (change between .year, .month, .day to execute the rebalancing)

# set initial shares on the first day
shares_df.loc[new1.index[0], :] = [np.floor((portfolio_value * w) / new1[s][0]) for s,w in zip(companies, weights)]

# initialize variables
balance_year = new1.index[0].year
signal = False
count = 0    # for loop count purpose

# Store previous values in a dictionary
prev_values = {}

# Calculate portfolio value for the first day
portfolio_value = sum([shares_df.loc[new1.index[0], s + '_shares'] * new1.loc[new1.index[0], s] for s in companies])

for day in new1.index:
    count += 1
    if day == new1.index[0]:
        shares_df.loc[day] = shares_df.loc[day] # First day

        # Store initial values as previous values
        for col in shares_df.columns:
            prev_values[col] = shares_df.loc[day, col]


    elif day.year != balance_year:
        signal = True
        # calculate new shares based on the new portfolio value and weights
        new_shares = [np.floor((portfolio_value * w) / new1[s][day]) for s,w in zip(companies, weights)]
        shares_df.loc[day, :] = new_shares
        balance_year = day.year
        count += 1
        # print(f'Rebalance: {day.date()}, count: {count}') # uncomment to debug days ;)
        # Store new values as previous values
        for col in shares_df.columns:
            prev_values[col] = shares_df.loc[day, col]

    else:

        signal = False

        # Use previous values if it is not a rebalancing date
        shares_df.loc[day, :] = [prev_values[col] for col in shares_df.columns]
        
        # print(f'Not rebalance, regular day: {day.date()}') # uncomment to debug days ;)



    # Calculate asset values and portfolio value for the current day
    asset_values = [shares_df.loc[day, s + '_shares'] * new1.loc[day, s] for s in companies]
    portfolio_value = sum(asset_values)
    
    new1.loc[day, 'Signal'] = signal
    new1.loc[day, 'Portfolio_Value'] = portfolio_value
    
    # Add shares to stock data frame
    for s in companies:
        new1.loc[day, s + '_shares'] = shares_df.loc[day, s + '_shares']
        new1.loc[day, s + '_value'] = shares_df.loc[day, s + '_shares'] * new1.loc[day, s]

In [ ]:
# Calculate log returns for portfolio
new1['Portfolio_Value_rets'] = np.log(new1['Portfolio_Value'] / new1['Portfolio_Value'].shift(1))

# Calculate log returns for each stock
for stock in companies:
    new1[f'{stock}_rets'] = np.log(new1[stock] / new1[stock].shift(1))

In [ ]:
new1

In [ ]:
start_date_benchmark = new1.index[0]
new1 = new1.dropna()

In [ ]:
new1

In [ ]:
# Calculate daily weight per asset
for s in companies:
    new1[s + '_weight'] = new1[s + '_value'] / new1['Portfolio_Value']

In [ ]:
new1.filter(regex='weight')

In [ ]:
import plotly.graph_objs as go
fig = go.Figure()

# Loop through each stock and add a trace for its shares
for stock in companies:
    fig.add_trace(go.Scatter(x=new1.index, y=shares_df[stock+'_shares'], mode='lines', name=stock+'_shares'))

fig.update_layout(title='Shares per day',
                  xaxis_title='Date',
                  yaxis_title='Shares',
                  width=800,
                  height=400)

fig.show()

In [ ]:
fig = go.Figure()

# Loop through each stock and add a trace for its shares
for stock in companies:
    fig.add_trace(go.Scatter(x=new1.index, y=new1[stock + '_weight'], mode='lines', name=stock + '_weight'))

fig.update_layout(title='Weights per day',
                  xaxis_title='Date',
                  yaxis_title='Weights',
                  width=1000,
                  height=600)

fig.show()


In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create subplot layout
fig = make_subplots(rows=2, cols=2, subplot_titles=('Portfolio Returns', 'Asset Returns', 'Shares Holding per Asset', 'Weights per Asset'))

# Add traces to the subplots
fig.add_trace(go.Scatter(x=new1.index, y=new1['Portfolio_Value_rets'].cumsum(), name='Portfolio'), row=1, col=1)

for s in companies:
    fig.add_trace(go.Scatter(x=new1.index, y=new1[f'{s}_rets'].cumsum(), name=f'{s}'), row=1, col=2)
    fig.add_trace(go.Scatter(x=shares_df.index, y=shares_df[f'{s}_shares'], name=f'{s}'), row=2, col=1)
    fig.add_trace(go.Scatter(x=new1.index, y=new1[f'{s}_weight'], name=f'{s}'), row=2, col=2)

# Update subplot layout
fig.update_layout(height=800, width=1200, title='Strategy Overview', showlegend=False)

# Display the plot
fig.show()

In [ ]:
# DISCRETE ALLOCATION OF EFFICIENT FRONTIER
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import pandas as pd

# Function to convert INR to USD
def inr_to_usd(amount_inr):
    # Assuming 1 USD = 75 INR
    return amount_inr / 75.0

portfolio_amount_inr = float(input("Enter the amount you want to invest (INR): "))

# Convert INR to USD
portfolio_amount_usd = inr_to_usd(portfolio_amount_inr)

if portfolio_amount_inr != '':
    # Get discrete allocation of each share per stock
    latest_prices = get_latest_prices(data2)
    weights = cleaned_weights
    discrete_allocation = DiscreteAllocation(weights, latest_prices, total_portfolio_value=int(portfolio_amount_usd))
    allocation, leftover = discrete_allocation.lp_portfolio()

    discrete_allocation_list = []

    for symbol in allocation:
        discrete_allocation_list.append(allocation.get(symbol))

    portfolio_df = pd.DataFrame(columns=['Ticker', 'Number of stocks to buy'])
    portfolio_df['Ticker'] = allocation
    portfolio_df['Number of stocks to buy'] = discrete_allocation_list

    # Convert leftover USD to INR
    leftover_inr = leftover * 75

    print('Number of stocks to buy with the amount of ₨', portfolio_amount_inr)
    print(portfolio_df)
    print('Funds remaining with you will be: ₨', int(leftover_inr))

In [ ]:
latest_prices.loc['Reliance Communications Limited']

In [ ]:
500-(2.0999999046325684*75*1)